In [1]:
import onnx
import onnx_tf
from onnx_tf.backend import prepare
import tensorflow as tf
from ultralytics import YOLO
import cv2
import numpy as np

2024-05-16 23:28:11.690302: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-16 23:28:11.711156: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 23:28:11.711180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 23:28:11.711806: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-16 23:28:11.715947: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Load YOLOv8 model
model = YOLO('best.pt')  # Replace with your model path

# Convert to ONNX
success = model.export(format='onnx') 

if not success:
    raise RuntimeError("ONNX export failed")


Ultralytics YOLOv8.2.9 🚀 Python-3.10.14 torch-2.3.0+cu118 CPU (AMD Ryzen 5 7600X 6-Core Processor)
Model summary (fused): 168 layers, 3007208 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'best.pt' with input shape (1, 3, 1024, 1024) BCHW and output shape(s) (1, 12, 21504) (6.0 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 0.7s, saved as 'best.onnx' (11.9 MB)

Export complete (2.4s)
Results saved to /mnt/c/Users/jason/GitHubRepos/LectorAI-TextExtraction
Predict:         yolo predict task=detect model=best.onnx imgsz=1024  
Validate:        yolo val task=detect model=best.onnx imgsz=1024 data=workspace/mix_up_data/YOLO_txt_main_sub/main_Dataset/data_main.yaml  
Visualize:       https://netron.app


In [4]:
# Load ONNX model
onnx_model = onnx.load('best.onnx')  # Replace with your ONNX file path

# Prepare TensorFlow model (includes weights)
tf_rep = prepare(onnx_model)

# Export TensorFlow SavedModel (includes weights)
tf_rep.export_graph("try/tf_model")


2024-05-16 23:28:56.045967: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.NEAREST_NEIGHBOR...)` instead.


INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_196_x, add_44_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: try/tf_model/assets


INFO:tensorflow:Assets written to: try/tf_model/assets
INFO:absl:Writing fingerprint to try/tf_model/fingerprint.pb


In [15]:
# Load TensorFlow SavedModel (with weights)
tf_model = tf.saved_model.load("try/tf_model")

# Make predictions
image = cv2.imread("data/Jason_Ad_2.jpg")
resized_image = cv2.resize(image, (1024, 1024)) # nathans size
resized_image = np.expand_dims(resized_image, axis=0)
resized_image = tf.convert_to_tensor(resized_image, dtype=tf.float32)
resized_image = tf.transpose(resized_image, perm=[0, 3, 2, 1])
output_dict = tf_model(images = resized_image)

# Process predictions (boxes, scores, classes)
boxes = output_dict['output0']
scores = output_dict['output1']
classes = output_dict['classes']

2024-05-16 23:38:19.754745: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:933] Skipping loop optimization for Merge node with control input: StatefulPartitionedCall/assert_equal_1/Assert/AssertGuard/branch_executed/_139


KeyError: 'output1'